<div style="text-align: center; font-weight: bold; font-size: 300%">Rendu PC6 MAP412</div>                                                        <br />      
<div style="text-align: center; font-size: 150%">École Polytechnique, MAP412 - octobre 2019</div><br />  
<div style="text-align: center; font-size: 120%">Jean-Charles Layoun</div>

In [1]:
import numpy as np

import scipy.sparse as scsp

import time

from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import row, column

from IPython.display import display, Math, Latex

from numpy.linalg import solve, cond, inv, det, eig, qr, norm
from scipy.linalg import lu_factor, lu_solve, lu, block_diag, solve_banded
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import splu

output_notebook(hide_banner=True)

# Conditionnement

Considérons l'exemple suivant :
$$
    A=\begin{pmatrix}
      10 & 1 & 4 & 0 \\
      1  & 10& 5 & -1\\
      4 & 5 & 10 & 7 \\
      0 & -1 & 7 & 9
    \end{pmatrix}
    \text{ avec }
    b_1 = \begin{pmatrix}
    15\\15\\26\\15
    \end{pmatrix}
    \text{ et }
    b_2 = \begin{pmatrix}
    16\\16\\25\\16
    \end{pmatrix}
$$

**Calculez le déterminant de $A$ et l'inverse de $A$ avec les fonctions *numpy.linalg.inv* et *numpy.linalg.det***

In [2]:
A = np.mat([[10, 1, 4, 0], [1, 10, 5, -1], [4, 5, 10, 7], [0, -1, 7, 9]])
b1 = np.array([15, 15, 26, 15])
b2 = np.array([16, 16, 25, 16])

print(np.linalg.inv(A))
print("\r")
print(np.linalg.det(A))
print("\r")
print(np.linalg.eig(A))
print("\r")

[[ 105.  167. -304.  255.]
 [ 167.  266. -484.  406.]
 [-304. -484.  881. -739.]
 [ 255.  406. -739.  620.]]

1.000000000001049

(array([5.34260914e-04, 1.91224791e+01, 8.99416974e+00, 1.08828169e+01]), matrix([[-0.23674149,  0.35684188, -0.83667786, -0.3414491 ],
        [-0.37692363,  0.38246091,  0.53571465, -0.65166099],
        [ 0.68605301,  0.71820543,  0.07646032,  0.08755599],
        [-0.57551135,  0.45887742,  0.08446173,  0.67162818]]))



**Résolvez les systèmes $Ax_1=b_1$ et $Ax_2 = b_2$ en utilisant soit l'inverse trouvée ci-dessus, soit *numpy.linalg.solve*. Expliquez le comportement obtenu et donnez une borne inférieure pour le conditionnement de la matrice $A$.**

In [3]:
x1 = solve(A,b1)
print(x1)
print("\r")
x2 = solve(A,b2)
print(x2)
print("\r")

borneInf = norm(x2-x1)*norm(b1)/norm(x1)/norm(b2-b1)
print("La valeur du conditionnement donnée par la borne inf : ",borneInf)
print("La valeur du conditionnement donnée par numpy : ", cond(A))


[1. 1. 1. 1.]

[  832.  1324. -2407.  2021.]

La valeur du conditionnement donnée par la borne inf :  32252.894883611825
La valeur du conditionnement donnée par numpy :  35792.39762897281


**Calculez le conditionnement de $A$ par les deux méthodes suivantes :**
* **calculez les valeurs propres de $A$ avec *numpy.linalg.eig* et utilisez le point précédent**
* **utiliser *numpy.linalg.cond***


# Décomposition QR et résolution de systèmes surdimensionnés

On considère le système surdimensionné $Ax=b$, avec 
$$
    A=\begin{pmatrix}
      10 & 1 & 4  \\
      1  & 3 & 5 \\
      4 & 5  & 10 \\
      2 & 5  & 6 \\
      1 & 8  & 6 \\
      5 & 9  & -4 \\
      0 & -1 & 7 
    \end{pmatrix}\ 
    \text{ et }\ 
    b = \begin{pmatrix}
    14\\15\\26\\28\\47\\70 \\-15
    \end{pmatrix}
$$

**Utiliser la méthode de la question du sujet de PC pour résoudre le système linéaire surdimensionné. On pourra utiliser la fonction *numpy.linalg.qr*.**

In [4]:
A=np.array([[10,1,4],[1,3,5],[4,5,10],[2,5,6],[1,8,6],[5,9,-4],[0,-1,7]])
b=np.array([14,15,26,28,47,70,-15])

Q, R = qr(A, 'complete')
#Q = np.mat(Q)
#R = np.mat(R)
#print(Q * R)

bp = np.dot(Q.T, b)[:3]
Rp = R[:3]
#print(bp)
#print(R[:3])
#print(Q.T @ Q)
#print(Q @ R)

x = solve(Rp, bp)
print("x = ",x)

display(Math(r"$\vert\vert Ax-b \vert\vert_2$ = {}".format(norm(A.dot(x)-b))))


x =  [ 1.210814    6.58773664 -1.18569127]


<IPython.core.display.Math object>

# Stockage de matrice et Laplacien

 ## Cas 1D

 ### Matrice pleine
 
On considère une matrice $A$ tridiagonale, *i.e.* dont les seuls coefficients non-nuls sont donnés par :
$A_{i,j} \neq 0 \quad\Rightarrow\quad |i-j| \le 1.$  

On écrit cette matrice $A$ sous la forme
  $$ A = \left( \begin{array}{cccccc}
    a_1    & b_1    & 0       &\dots   & 0 \\
    c_2    & a_2    & b_2     & \ddots& \vdots  \\
    0      & \ddots & \ddots &\ddots  &0\\
    \vdots & \ddots & \ddots &\ddots  & b_{N-1}\\
    0      & \dots  &  0   &   c_N  & a_N
  \end{array}\right).$$
  
  
Pour les applications numériques, on fixera :
\begin{equation} a_i = \frac{-2}{h^2}, \quad b_i = \frac{1}{h^2} \quad \text{ et } c_i = \frac{1}{h^2} \text{ avec } h = \frac{1}{N}, \quad N = 2048.
\end{equation}

Ici $h$ est un pas d'espace et $N$ un nombre de mailles. Cette matrice correspond à une discrétisation de l'opérateur $-\Delta$, dont la construction sera vue plus tard dans le cours.

**On utilisera les valeurs numériques ci-dessus. Utiliser la fonction *scipy.linalg.lu_factor* pour construire deux matrices $L$ et $U$ respectivement triangulaire inférieure et supérieure telles que $A=LU$.**

In [5]:
N = 2048
h = 1/(N+2)
d = [-2/(h**2) for i in range (0,N)] #ai
d1 = [1/(h**2) for i in range (0,N-1)]#ci et bi

diagonals = [d, d1, d1]

A = np.eye(N,N) *(-2/(h**2))
A[np.arange(N-1), np.arange(N-1)+1] = d1
A[np.arange(N-1)+1, np.arange(N-1)] = d1
#print(len(A[0]))
#print(A)

LU, piv = lu_factor(A)


**Résoudre $-Au=f$ avec $f_i = x_i^3$ où $x_i = ih$ en utilisant la factorisation LU précédente et la fonction *scipy.linalg.lu_solve*.**

In [8]:
x = [i*h for i in range (1, N+1)]
f = [-xi**3 for xi in x]

t1= time.time()
u = lu_solve((LU, piv), f)
t_resol_pleine = time.time()-t1

print("Test pour but de savoir si u est bien solution du système, valeur du test : \n\n\t\t\t\t\t",
      np.allclose(A @ u - f, np.zeros((N,))))

Test pour but de savoir si u est bien solution du système, valeur du test : 

					 True


**Calculer la solution analytique $x\mapsto u(x)$ du problème $-\Delta u = f$ et tracer les courbes $u(x_i)$ et $u_i$ en fonction de $x_i$.** <br>
***


##### Solution théorique :
$$ u(x) = \cfrac{x}{20} \left(1 - x^4 \right)$$<br>

In [9]:
def fu(x):# solution théorique
    return x*(1 - x**4)/20

u_th = [fu(xi) for xi in x]


fig = figure(title="Superposition de la solution théorique et numérique :", width=500, height=350)
fig.line(x, u_th, color="blue", line_width=2, legend="Solution théorique", line_dash= [5,10])
fig.line(x, u,  color="red", legend="Solution numérique")
show(fig)

### Matrice tridiagonale

**Implémenter l'algorithme de décomposition LU basé sur un stockage tridiagonal de la matrice, c'est-à-dire qu'on ne stocke que les composantes non-triviales des matrice $A$, $L$ et $U$. On écrira pour cela :**

\begin{gather}
      \tilde{A} = \left( \begin{array}{c|c|c|c} 0 & b_{1} & \dots & b_{N-1}\\ \hline  a_{1} & a_{2} & \dots & a_{N} \\ \hline c_{2} & c_{3} & \dots  & 0 \end{array} \right) = \left( \begin{array}{l} b \\ a \\ c \end{array}\right) \in \mathbb{R}^{3,N}, \nonumber \\
      \tilde{L} = \left( \gamma_{2}, \gamma_{3}, \dots, \gamma_{N}, 0 \right) \in\mathbb{R}^{1,N}, \qquad
      \tilde{U} = \left( \begin{array}{c|c|c|c|c} 0 & \beta_2 & \beta_3 & \dots & \beta_N \\ \hline \alpha_{1} & \alpha_{2} & \alpha_{3} & \dots & \alpha_{N} \end{array} \right) \in\mathbb{R}^{2,N}.
      %\tilde{A} = \left( \begin{array}{c|c|c} 0 & A_{1,1} & A_{1,2} \\ A_{2,1} & A_{2,2} & A_{2,3} \\ \vdots & \vdots & \vdots \\  A_{N,N-1} & A_{N,N} & 0 \end{array} \right) \in \mathbb{R}^{3,N}, \nonumber \\
      %\tilde{L} = \left( \begin{array}{c} 0 \\ L_{2,1} \\ L_{3,2} \\ \vdots \\ L_{N,N-1}\end{array}\right) \in\mathbb{R}^{N,1}, \qquad
      %\tilde{U} = \left( \begin{array}{c|c} U_{1,1} & U_{1,2} \\ \vdots & \vdots \\ U_{N-1,N-1} & U_{N-1,N} \\ U_{N,N} & 0\end{array} \right) \in\mathbb{R}^{N,2}.
        %= (0, \ A_{1,1},\ A_{1,2}), \quad \tilde{A}_{1,i} = (A_{i,i-1}, \ A_{i,i}, \ A_{i,i+1}), \quad \tilde{A}_{N,:} = (A_{N,N-1},\ A_{N,N}, \ 0).
\end{gather}

In [10]:
def LU_modif(A):# N'accepte que des matrices tridiagonales carrées de type n*n où n>=3
    
    n, m = len(A),len(A[0])    

    if(n<=2):
        raise Exception("La matrice n'est pas bonne ! ", A)
    if(n!=m):
        raise Exception("La matrice n'est pas carré !", A)
    
    a, c, b = np.diag(A, 0), np.diag(A, -1), np.diag(A, 1)
    A_th = np.diag(a,0) + np.diag(c,-1) + np.diag(b,1) # Construction de ce que devrait être une matrice tridiag
    
    if(np.allclose(A - A_th, np.zeros((n,))) != True):
        raise Exception("La matrice n'est pas tridiag !", A)
    
    Beta = [bi for bi in b] # Beta_i = b_i
    Alpha = [a[0]]
    Gamma = []
    
    for i in range(0,n-1):
        Gamma += [c[i]/Alpha[i]]
        Alpha += [a[i+1] - Gamma[i]*Beta[i]] 
    Gamma += [0]
    L = np.array(Gamma)
    Beta = [0] + Beta
    U = np.array([Beta, Alpha])
    
    c = np.array(0) + c
    A_tilde = np.array([np.array(Beta), a, c])
    
    return A_tilde, L, U
    
    

**Tester votre algorithme sur la matrice :**

\begin{equation} 
A = \left(\begin{array}{ccc} 2 & 1 & 0 \\ -1 & 3 & 1 \\ 0 & 1 & 4 \end{array}\right).
\end{equation}

In [79]:
A = np.array([[2, 1, 0], [-1, 3, 1], [0, 1, 4]])

try:
    A_tilde, L, U = LU_modif(A)
except AssertionError as error :
    print(error)

display(Math(r"\tilde A = {0} ".format( A_tilde)))
display(Math(r"\tilde L = {0}".format(L)))
display(Math(r"\tilde U = {0}".format(U)))

#P_t, L_t, U_t = lu(A)
#print(L_t, "\n", U_t)
L_rec = np.diag(L[:-1], -1) +np.eye(3,3)
U_rec = np.diag(U[0][1:],1) + np.diag(U[1],0) 
print("Reconstitution de A ave LU : \n",L_rec @ U_rec)#Modèle de stockage est bon!

##Je test le modele de test que j'ai conçu pour reconnaître les matrices tridiag
#print(np.diag(A, 0), np.diag(A, -1), np.diag(A, 1))
#A_th = np.diag(np.diag(A,0),0) + np.diag(np.diag(A,-1),-1) + np.diag(np.diag(A,1),1)
#print(A_th)
#print(np.allclose(A - A_th, np.zeros((3,))))


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Reconstitution de A ave LU : 
 [[ 2.  1.  0.]
 [-1.  3.  1.]
 [ 0.  1.  4.]]


**Adapter et implémenter les algorithmes de remontée et de descente adaptés aux matrices triangulaires stockées sous forme tridiagonale. Tester votre algorithme pour la résolution de $A u = -f$ avec la matrice $A$ de question précedente et $f = (1,\ 1,\ 1)^T$, en utilisant la décomposition $LU$ de $A$ obtenue à la question précédente.**

In [80]:
f =[-1, -1, -1]


def Solve_modif(L,U,f):
    ##On résoud d'abord LY = f
    N = len(L)
    Y = [0 for i in range (0,N)]
    Y[0]= f[0]
    for i in range (1,N):
        Y[i] = f[i] - L[i-1]*Y[i-1]
        
    ## Puis on résoud UX = Y, c'est une remonté de bas en haut, contrairement à Y
    X = [0 for i in range (0,N)]
    X[-1]= Y[-1]/U[1][-1] # X_n = f_n/alpha_n
    for i in range (1,N):
        X[-i-1] = (Y[-i-1] - U[0][-i]*X[-i])/ U[1][-i-1]    
    return X

A = np.array([[2, 1, 0], [-1, 3, 1], [0, 1, 4]])

t1 = time.time()
try:
    A_tilde, L, U = LU_modif(A)
except AssertionError as error :
    print(error)
    
X = np.array(Solve_modif(L, U, f))

t_resol_tridiag = t1 - time.time()

print("Test pour but de savoir si u est bien solution du système, valeur du test : \n\n\t\t\t\t\t",
      np.allclose(A @ X - f, np.zeros((3,))))

Test pour but de savoir si u est bien solution du système, valeur du test : 

					 True


**Résoudre le système $Au = -f$ avec la matrice $A$ issue de la discrétisation de l'opérateur $-\Delta$ . Comparer la précision des résultats et les temps d'execution (en utilisant *time.time*) avec ceux obtenus avec la matrice pleine. Commenter les avantages et inconvénients de ce stockage.**

In [82]:
print("Temps de résolution pour une matrice pleine 1D : ", t_resol_pleine)
print("\nTemps de résolution pour le stockage en matrice tridiag : ", t_resol_tridiag)

Temps de résolution pour une matrice pleine 1D :  0.0029916763305664062

Temps de résolution pour le stockage en matrice tridiag :  0.0


##### Remarques :
<ol>
<li>Les calculs de la méthode tridiag sont plus rapides mais la précision est la même.</li>
<li>Stockage inutile pour des petites matrices. On optimise grandement le stockage en mémoire en utilisant le stockage tridiagonale pour des matrices de grande taille et tridiagonale.</li>
<li>Un désavantage serait de recoder la résolution de système...</li>
</ol>

## Cas 2D

On considère maintenant une matrice bande, *i.e.* dont les coefficients satisfont si $|i-j| < K$ alors$A_{i,j} = 0,$ où $K>0$ est appelée la largeur de bande.

  On utilisera pour les applications numériques la matrice $A$ définie par :
  
\begin{align}
    A &= \left( \begin{array}{c|c|c|c}
      D    & I/h^2    &\dots   & 0 \\ \hline
      I/h^2    & D    & \ddots& \vdots  \\ \hline
      \vdots   & \ddots &\ddots  &I/h^2\\ \hline
      0      & \dots  &   I/h^2  & D
    \end{array}\right), \quad %\in\mathbb{R}^{N^2\times N^2}, \
    D = \left( \begin{array}{cccc}
      -4/h^2    & 1/h^2    & \dots   & 0 \\
      1/h^2    & -4/h^2    & \ddots& \vdots  \\
      0      & \ddots & \ddots  &1/h^2\\
      0      & \dots  &  1/h^2  & -4/h^2
    \end{array}\right) \in\mathbb{R}^{N\times N},
    \label{Lapl_2D}
\end{align}

où $I$ est la matrice identité de taille $N=64$ et $h = \frac{1}{N+2}$. Cette matrice correspond à une discrétisation de l'opérateur $-\Delta$ en 2D.

### Matrices pleines

**Je suis la covention 2^10 bits = 1 Kilo Bites = 1Kb**.

***
##### Commentaires :

<ol>
<li>Le temps d'éxécution est assez long, 1.8 s en moyennee.</li>
<li>La matrice A est de taille $N^2 \times N^2$ avec N = 64 = $2^6$, soit $2^{24}$ flotants stockés dans la matrice A. Or les flottants sont codés sur 64 bits par défaut $\Rightarrow$ la matrice prend en mémoire RAM $2^{30} bits$, soit <b>1 Gb.</b> </li>
<li>La matrice est <b>très creuse</b>, c'est pour cela que nous envisageons dans les prochaines questions de réduire le nombre de zéros que nous enregistrons. Il y a $5N^2 -4N$ valeurs non nul (les 5 diagonales) sur $N^4$ valeurs stockées.</li>
</ol>


In [13]:
N = 64 
h = 1/(N+2)

t1 = time.time()

I = np.eye(N)/(h**2)

d1 = [1/(h**2) for i in range (0,N-1)]
d  = [-4/(h**2) for i in range (0,N)]
D = np.diag(d,0) + np.diag(d1,-1) + np.diag(d1,1)


##Construction de A manuellement:
A = np.zeros((N**2,N**2))

for i in range(0, N**2):
    for j in range((i//N-1)*N,(i//N+2)*N):
        if(j<0):
            a=0
        elif(j>=N**2):
            break
            
        elif((i//N-1)*N <= j < i//N*N):
            A[i][j] = I[i%N][j%N]
        
        elif(i//N*N <= j < (i//N+1)*N):
            A[i][j] = D[i%N][j%N]
            
        elif((i//N+1)*N <= j < (i//N+2)*N):
            A[i][j] = I[i%N][j%N]


x = [i*h for i in range(1,N+1)]
y = [i*h for i in range(1,N+1)]
fi = [-(x[i]**3 + y[i]**3) for i in range (0, N) ]

f = []
for i in range (0,N):
    f += fi

    
u = solve(A,f)

t2 = time.time()
f = np.array(f)

print("On trouve la solution en {} s\n".format(t2-t1))
print(u)
## La matrice stock

On trouve la solution en 1.6364178657531738 s

[3.18511192e-05 6.38265061e-05 9.60499897e-05 ... 1.63901243e-03
 1.25201956e-03 7.34946405e-04]


### Matrices bandes

On stocke désormais la matrice sous forme bande, c'est-à-dire qu'on écrit :

\begin{align*}
      \tilde{A} = \left(
      \begin{array}{c|c|c|c|c|c}
        0             & \dots & 0        & A_{1,K+1} & \dots   & A_{N-K,N}\\
        \hline \vdots &       &  & A_{2,K+1} &\dots   & A_{N-K+1,N}\\
        \hline 0      & A_{1,2} &          &          &         & \vdots    \\
        \hline A_{1,1} & A_{2,2} & \dots  & \dots     & \dots   & A_{N,N} \\
        \hline A_{2,1} & A_{3,2} & \dots   & \dots    & A_{N,N-1} & 0 \\
        \hline \vdots &        &         &          &         & \vdots \\
        \hline A_{K+1,1} & \dots & A_{N,N-K} & 0       & \dots   &  0
      \end{array} \right) \in \mathbb{R}^{2K+1,N^2}.
\end{align*}

##### Commentaires :

<ol>
<li>On remarque que K = N parce que la dernière diagonale non nulle est la N$^{ème}$.</li>
<li>Le temps d'éxécution est beaucoup moin long, 0.02 s en moyennee comparer à 1.8 s.</li>
<li>La matrice $\tilde A$ est de taille $(2N+1) \times N^2$ avec N = 64 = $2^6$, soit $2^{19} + 2^{12}$ flotants stockés dans la matrice A. Or les flottants sont codés sur 64 bits par défaut $\Rightarrow$ la matrice prend en mémoire RAM $2^{25} + 2^{18} bits$, soit <b>32.25 Mb</b>. </li>
<li>La matrice reste <b>creuse</b>. Il y a cette fois-ci $5N^2 -4N$ valeurs non nulles (les 5 diagonales) sur $2N^3 + N^2$ valeurs stockées. Moin qu'avant certes, mais on peut mieux faire.</li>
</ol>


In [46]:
def A_banded(A, N, h):
    #Une erreur dans la donnée, A tilde est dans M2k+1,n^2(R), sinon la sol n'est pas de la bonne dimension
    A_tilde = np.zeros((2*N+1,N**2)) 
    
    
    
    # la bande est de largeur N
    c = 0
    ## On définit la Nième et -Nieme diagonale
    d = [1/(h**2) for i in range (0,N**2 - N)]
    
    A_tilde[0][N:] = d
    A_tilde[2*N][:(N**2-N)] = d
    
    ## La diagonale de la matrice
    
    d = [-4/(h**2) for i in range (0,N**2)]
    A_tilde[N] = d[:(N**2)]
    
    ## On s'attaque aux diagonales -1 et 1
    
    d = [1/(h**2)*(0 if i%N==(N-1) else 1) for i in range (0,N**2)]

    A_tilde[N-1][1:] = d[:(N**2 - 1)]
        
    A_tilde[N+1][:(N**2-1)] = d[:(N**2 - 1)]
    
    return A_tilde

t1 = time.time()
A_tilde = A_banded(A, N, h)
u_banded = solve_banded((N,N), A_tilde, f)
t2 = time.time()

print("On trouve la solution en {} s\n".format(t2-t1))
print(u_banded)

On trouve la solution en 0.018950223922729492 s

[3.18511192e-05 6.38265061e-05 9.60499897e-05 ... 1.63901243e-03
 1.25201956e-03 7.34946405e-04]


### Matrices creuses CSR

**Bonus. Construire la matrice $A$ de la discrétisation de l'opérateur $-\Delta$ en format CSR (utiliser scipy.sparse.csr matrix) avec Scipy (https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html).**

In [64]:
t1 = time.time()
A_csr = csr_matrix(A)
u_csr = splu(A).solve(f)
t2 = time.time()

t= t2-t1

#print(A_csr, "\n")
## Elle stock les indices i et j des valeurs non nuls de A ainsi que la valeur associé.

print("Nombre de valeurs non nuls est : ", np.count_nonzero(A))

Str = "   {}".format(5*N**2-4*N)

display(Latex("\nIl est bien égale à $5N^2-4N$ =    "+ Str))

Nombre de valeurs non nuls est :  20224


<IPython.core.display.Latex object>

##### Commentaires :

<ol>
<li>Le temps d'éxécution est un peu plus long que celui des matrices bandes, 0.19 s en moyennee comparer à 0.02 s.</li>
<li>Ceci dit, la matrice CSR de A est de taille $3 \times (5N^2 -4N)$ soit 60672 flotants stockés dans la matrice CSR de A $\Rightarrow$ la matrice prend en mémoire RAM <br>3 883 008 $bits$, soit <b>3.7 Mb</b>. </li>
<li>La matrice reste <b>creuse</b>. Il y a cette fois-ci 3 fois plus de valeurs stockées que de valeurs non nulles. </li>
<li>Le stockage est donc de l'ordre de $N^2\times  2^6$, une amélioration conséquente % aux matrices bandes. Ce qui pourrait défois justifié l'utilisation de la matrice CSR lorsqu'on est réstraint en mémoire et le temps de calcul n'est pas une contrainte.</li>
<li>Malgré avoir éliminer les 0, il reste beaucoup de redondance dans la matrice, finalement il n'y a que 3 valeurs différentes. On pourrait modifier le stockage CSR, en stockant seulement les 3 différentes valeurs et leurs emplacements respectifs. On peut aussi imaginer une manière de stocker les redondances dans le même esprit d'algorithme de ZIP. On fera plus de calcul, mais on utilisera beaucoup moin de mémoire!</li>    
</ol>


In [47]:
u_csr = splu(A).solve(f)

print("On trouve la solution en {} s\n".format(t))

print(u_csr)

On trouve la solution en 0.19211649894714355 s

[3.18511192e-05 6.38265061e-05 9.60499897e-05 ... 1.63901243e-03
 1.25201956e-03 7.34946405e-04]
